# SSH keys

## Overview:
- **Teaching:** 15 min
- **Exercises:** 0 min

**Questions**
- How can I generate ssh keys for use with HPC and other remote servces
- How does the use of ssh kys improve the security of remote connections

**Objectives**
- Know that ssh_keys are an alternative to password authentication that can be used to increase the security of remote connections
- Understand how to generate **secure** ssh keys

## Information: What is a public key pair?

A public key pair consists of two parts, a public part and a private part which are related.  One is used to lock the message the other is used to unlock the message:

* The public part is used to lock (encrypt) the message so that it can be sent over the internet and can be shared.
* The private part is used to unlock (decrpyt) the message and should not be shared.

## Set up an SSH key pair *protected by a passphrase*

Some systems will also/instead require you to set up an SSH key pair to access the services.  The SSH key pair consists of a private part and a public part.  The public key can be put on remote machines to allow you to log-in without the use of a password.  You keep the private part of the key secure on your local machine protected with a passphrase.

Public key encryption uses fancy maths to enable secure communication over an open channel.  There are a number of methods the most common being [RSA](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) which uses prime numbers.  If you are going to use this you should use a key size of at least 2048 and preferably 4096.  Public keys can be broken with brute force computation and the longer the key the more secure it is.

An alternative encryption method and the one we recommend you to use are based on [EdDSA](https://en.wikipedia.org/wiki/EdDSA) (Ed25519).  For our purposes the key pair works in the same way.  Your public key can go on the remote resource or service and the private key is kept protected on your local machine.  You can generate a key pair with:

```bash
ssh-keygen -o -a 100 -t ed25519 -f ~/.ssh/id_ed25519
```

* `ssh-keygen` is the command to generate the key pair
* `-o` specifies to use a strong format to save the key
* `-a 100` increases the strength of encryption with your passphrase
* `-t ed25519` specifies the encryption method used
* `-f filename` specifies the name of the ssh key, by default these are stored in the directory `~/.ssh`

When you create a SSH key pair you will be prompted to provide a `passphrase`.  This is effectively password for your private key and like a password should be kept secret.  Now when you try to use the key, you should be asked for your key pair **passphase** (which you entered when you created the key pair) rather than your remote machine **password**.

## Windows

If you have a Windows machine then you can create keys with:

* Putty, use puttygen
* MobaXterm, use MobaKeyGen

They use essentially the same interface to generate keys with these [instructions](https://docs.computecanada.ca/wiki/Generating_SSH_keys_in_Windows/en).

## Exercise: Generate an ssh key

### Linux:

Using the above command on your **local machine** generate an ssh key.

### Windows:

By following the instructions above or for gui interface at the [puttygen website](https://www.puttygen.com/) generate an ssh key.

## Information: Use passphrases

*If you do not use a passphrase then if someone gets hold of your private key they will be able to use your key to log in on any machine where you use that key.*

On systems or services you may not be able to use the `EdDSA` encryption.  In this case we recommend that you should use RSA keys of length 4096 (and at least 2048):

```bash
ssh-keygen -o -a 100 -t rsa -b 4096
```

## Information: Private keys are private

While it is necessary to share you public key in order to use public key encryption, the private key should never be shared or stored on remote services even though it is protected with a **passphrase**.

## Exercise: Sharing keys

The use of ssh keys is great for restricting access to users who have registered keys on the system, but would is the consequence if someone else gets hold of your keys.  What would happen if you were to use someone else's key?

[Solution]()

## Solution: Sharing keys

Entries in the authorized\_keys allows anyone with the matching private key to log in to the remote service with your credentials.  Therefore if someone can access your private key they can access any systems you use it on.  This is why we protect our keys with a passphrase.  

Similarly if you add someone else's public key you are effectively handing them your identity and password to your account on a remote system.

1. ***Always use a passphrase***
2. ***Never share keys, just like you never share passwords*** 

## Information: Key permissions

SSH requires that the private key has permissions:

```bash
-rw------- 1 user group  464 May 14 23:31 id_ed25519
```

This should be set correclly by default when you generate the key pair. If this changes for any reason SSH will reject your key when you attempt to use it.  You can correct the permissions with `chmod go-rwx -R ~/.ssh`, but should also make sure you understand why it isn't correct.

## Key pairs for multiple services

So far we have generated a single key with a default name for one service.  Using strong keys means that the key should secure but what happens if the key is compromised?  An intruder can now access all systems on which we use this key.  Therefore it is good practice to use a different key for each service you use. In order to do this you need to specify the name of key file:

```bash
ssh-keygen -o -a 100 -t ed25519 -f ~/.ssh/id_ed25519_service
```

Before when we connected to the remote machine `ssh` automatically tried default keys it found in `~/.ssh`. We can specify that we wish to use a specific key with:

```bash
ssh -i ~/.ssh/id_ed25519_service [userID]@<hpc-service>
```

However we now have an issue that we need to remember and specify the key we want to use for each service and typeout a longer command each time we want to connect to remote machines.  We can simplify this by adding the Host, and key file to our **ssh config**.  Edit `~/.ssh/config` and add/include:

```bash
Host <hpc-service>
    IdentityFile ~/.ssh/id_ed25519_service
```

Now when we connect to the service:

```bash
ssh [userID]@<hpc-service>
```

## Information: How many keys?

Having too many keys can become confusing so as an example I have a key for:

1. our local HPC service
2. local university systems
3. github
4. national/Tier 2 services
5. personal use

## Exercise: Generate a second key for github

Generate yourself a second key for use on github.

## Key Points:

- the use of ssh keys restrict access to users with public keys registered on the remote service
- **passphrases** must be used to protect private keys
- we recommend using the ed25519 key type but make sure this is supported by your service